In [1]:
from typing import List

import networkx as nx
from networkx import diameter, is_connected
from tqdm import tqdm

from rumor_centrality.evaluation import hop_distances
from rumor_centrality.graph_clustering import assign_all_nodes_cluster, visualise_cluster_graph, multiple_rumor_source_prediction
from rumor_centrality.graph_generator import small_world
from rumor_centrality.graph_simulations import si
from rumor_centrality.experiment import multiple_sources_experiment

In [2]:
experiment_params = {
    "max_infected_nodes": [100, 1000],
    # "node_count": 3000,
    "infection_prob": 0.3,
    "num_infection_centers": [2, 3, 5, 7, 10],
    "exp_iterations": 10,
}

In [3]:
from functools import partial
from rumor_centrality.graph_generator import synthetic_internet, scale_free, us_power_grid, internet

# Available Graphs
# Making graphs bigger, so that infections make sense on it
graph_types = {
    # "synthetic_internet_100": lambda: synthetic_internet(1000),
    "synthetic_internet_10000": partial(synthetic_internet,10000),
    # "scale_free_100": lambda: nx.Graph(scale_free(1000)),
    "scale_free_10000": partial(scale_free, 10000),
    "us_power_grid": us_power_grid,
    "internet": internet,
}

In [4]:
from itertools import product
from multiprocessing import Pool, TimeoutError
from rumor_centrality.experiment import multiple_sources_experiment

def unpack_result(result):
    try:
        return_value = result.get(timeout=250)
    except TimeoutError:
        return_value = None

    return return_value

In [ ]:
result = []

num_infection_centers = experiment_params["num_infection_centers"]
max_infected_nodes = experiment_params["max_infected_nodes"]
infection_prob = experiment_params["infection_prob"]
exp_iterations = experiment_params["exp_iterations"]

for num_infection_center, max_inf_nodes, graph_name in tqdm(list(product(num_infection_centers, max_infected_nodes, graph_types))):
    graph = graph_types[graph_name]

    with Pool(processes=10) as pool:
        multiple_results = [pool.apply_async(multiple_sources_experiment, (
            num_infection_center,
            infection_prob,
            max_inf_nodes,
            graph,
            graph_name,
        )) for i in range(exp_iterations)]
        result.extend([unpack_result(res) for res in multiple_results])

  0%|          | 0/40 [00:00<?, ?it/s]